https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_agents

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "http://localhost:8000"

In [2]:
from langchain.agents import load_tools

tool_names = ["python_repl"]
tools = load_tools(tool_names)
print(tools)

[PythonREPLTool(name='Python REPL', description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, python_repl=PythonREPL(globals={'__name__': 'langchain.tools.python.tool', '__doc__': 'A tool for running python code in a REPL.', '__package__': 'langchain.tools.python', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x10b5a10d0>, '__spec__': ModuleSpec(name='langchain.tools.python.tool', loader=<_frozen_importlib_external.SourceFileLoader object at 0x10b5a10d0>, origin='/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/langchain/tools/python/tool.py'), '__file__': '/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/langchain/tools/python/tool.py', '__cached__': '/Users/to.watanabe/worksp

In [3]:
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool
from langchain.agents import load_tools

tool_names = ["python_repl"]
tools = load_tools(tool_names)

llm = OpenAI(model_name="text-davinci-003")
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

tools.append(
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
)
print(tools)
print(len(tools))

[PythonREPLTool(name='Python REPL', description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, python_repl=PythonREPL(globals={'__name__': 'langchain.tools.python.tool', '__doc__': 'A tool for running python code in a REPL.', '__package__': 'langchain.tools.python', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x10b5a10d0>, '__spec__': ModuleSpec(name='langchain.tools.python.tool', loader=<_frozen_importlib_external.SourceFileLoader object at 0x10b5a10d0>, origin='/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/langchain/tools/python/tool.py'), '__file__': '/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/langchain/tools/python/tool.py', '__cached__': '/Users/to.watanabe/worksp

/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [4]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import DuckDuckGoSearchRun

llm = OpenAI(model_name="text-davinci-003")

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)

search = DuckDuckGoSearchRun()
tools.append(
    Tool(
        name="duckduckgo-search",
        func=search.run,
        description="useful for when you need to search for latest information in web"
    )
)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("""
現在の20代の日本人男性の平均身長を教えて。
そして、私の身長は168cmなため、日本全国から見た時の差を2乗した結果を教えて。
""")



> Entering new AgentExecutor chain...
 I should first look for the average height of men in their 20s in Japan.
Action: duckduckgo-search
Action Input: "Average height of men in their 20s in Japan"

/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/duckduckgo_search/compat.py:20: UserWarning: ddg is deprecated. Use DDGS().text() generator
  warnings.warn("ddg is deprecated. Use DDGS().text() generator")
/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/duckduckgo_search/compat.py:22: UserWarning: parameter time is deprecated, use parameter timelimit
  warnings.warn("parameter time is deprecated, use parameter timelimit")
/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/duckduckgo_search/compat.py:24: UserWarning: parameter page is deprecated, use DDGS().text() generator
  warnings.warn("parameter page is deprecated, use DDGS().text() generator")
/Users/to.watanabe/workspace/langchain_study/.venv/lib/python3.11/site-packages/duckduckgo_search/compat.py:26: UserWarning: parameter max_results is deprecated, use DDGS().text()
  warnings.warn("parameter max_results is deprecated, use DDGS().text(


Observation: What is the historical background of the Japanese folk's height, and how has it shifted so dramatically over time? What are the challenges of becoming tall in Japan? This blog post addresses these and other concerns. Individual's Height Throughout History In Japan Yedo ( old Tokyo) in Edo period The expected average height for a female should be 163 cm (5 ft 4.3 in), and the expected average height for a male should be 176.5 cm, based on the World Health Organization's (WHO) growth reference standards. The male, however, somehow falls short of the global average height. In Japan, the typical height is 158 centimetres (5'2″) for women and 172 cm (5'7″) for males. Average Height And Weight Of People In Japan Over Different Age Groups Japan average height and weight is different for all ages! Multiple aspects of a person's environment influence their development throughout their formative years. Japan's dietary supplement industry has a short history since they were not wide

'168 cmは172 cmと比べて4 cm短く、0.393701インチであるという結果です。'

In [5]:
from langchain.llms import OpenAI
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool

llm = OpenAI(model_name="text-davinci-003")
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True,
)
agent_executor.run("5つ目の素数は何？")



> Entering new AgentExecutor chain...
 I need to use a loop to find the 5th prime number
Action: Python REPL
Action Input: primes = [2, 3]

i = 2
while len(primes) < 5:
    i += 1
    for prime in primes:
        if i % prime == 0:
            break
    else:
        primes.append(i)
Observation: 
Thought: I now know the 5th prime number
Final Answer: 11

> Finished chain.


'11'

In [6]:
from langchain.llms import OpenAI
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool

llm = OpenAI(model_name="text-davinci-003")
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True,
)
agent_executor.run("""
Pytorchで単層ニューラルネットワークを作ってください。
この時、「-10~10」を定義域としたy=2xのデータを30個生成し、そのデータで100 epoch学習してください。
最後に、そのモデルを用いて、x=5に対する予測を行ってください。
""")



> Entering new AgentExecutor chain...
 Pytorchでニューラルネットワークを作成し、データを生成して、学習して、x=5に対する予測を行う
Action: Python REPL
Action Input: 
import torch
import torch.nn as nn
import torch.nn.functional as F

# define data
x_data = torch.linspace(-10, 10, 30).reshape(-1, 1)
y_data = 2*x_data

# define model
model = nn.Linear(1, 1)

# training
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(100):
    pred = model(x_data)
    loss = F.mse_loss(pred, y_data)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# prediction
x_test = torch.Tensor([[5]])
y_test = model(x_test)

Observation: ModuleNotFoundError("No module named 'torch'")
Thought: torchをインストールする
Action: Python REPL
Action Input: pip install torch
Observation: SyntaxError('invalid syntax', ('<string>', 1, 5, 'pip install torch\n', 1, 12))
Thought: コマンドプロンプトからインストールする
Action: Command Prompt
Action Input: pip install torch
Observation: Command Prompt is not a valid tool, try another one.
Thought: コマン

'Agent stopped due to iteration limit or time limit.'